In [ ]:
import heapq
import copy
import time
from collections import deque
from queue import Queue
import pygame
#!pip install xvfbwrapper
#!pip install pyvirtualdisplay
#import os
#from pyvirtualdisplay import Display
#os.environ["SDL_VIDEODRIVER"] = "dummy"

#import pygame
#pygame.init()
#screen = pygame.display.set_mode((300, 300))
#pygame.display.set_caption("Sudoku Puzzle")
#font = pygame.font.Font(None, 36)



def get_neighbors(state):
    neighbors = []
    i, j = next((i, j) for i, row in enumerate(state) for j, val in enumerate(row) if val == 0)
    for x, y in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if 0 <= i + x < 3 and 0 <= j + y < 3:
            new_state = copy.deepcopy(state)
            new_state[i][j], new_state[i + x][j + y] = new_state[i + x][j + y], new_state[i][j]
            neighbors.append(new_state)
    return neighbors

def manhattan_distance(state,goal_state):
    distance = 0
    goal = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                x, y  = divmod(state[i][j] - 1, 3)
                l , m = divmod(state[i][j] -1 , 3)
                distance += abs(x - l) + abs(y - m)
    return distance

def euclidean_distance(state,goal_state):
    distance = 0
    goal = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                x, y = divmod(state[i][j] - 1, 3)
                l, m = divmod(goal[i][j] - 1,3)
                distance += ((x - l) ** 2 + (y - m) ** 2) ** 0.5
    return distance


def A_star(initial_state, goal_state,heurisitc):
    heap = [(0, initial_state)]
    visited = set()
    parents = dict()
    g_score = {tuple(map(tuple, initial_state)): 0}
    f_score = {}
    if heurisitc == 1:
        f_score = {tuple(map(tuple, initial_state)): manhattan_distance(initial_state, goal_state)}
    elif heurisitc == 2:
        f_score = {tuple(map(tuple, initial_state)): euclidean_distance(initial_state, goal_state)}


    while heap:
        current_f, current_state = heapq.heappop(heap)
        if current_state == goal_state:
            path = []
            while current_state != initial_state:
                path.append(current_state)
                current_state = parents[tuple(map(tuple, current_state))]
            path.append(initial_state)
            path.reverse()
            return path, g_score[tuple(map(tuple, goal_state))], len(visited)

        visited.add(tuple(map(tuple, current_state)))
        for next_state in get_neighbors(current_state):
            next_g = g_score[tuple(map(tuple, current_state))] + 1
            if tuple(map(tuple, next_state)) not in visited or next_g < g_score.get(tuple(map(tuple, next_state)),
                                                                                    float('inf')):
                parents[tuple(map(tuple, next_state))] = current_state
                g_score[tuple(map(tuple, next_state))] = next_g
                if heurisitc == 1:
                    f_score[tuple(map(tuple, next_state))] = next_g + manhattan_distance(next_state, goal_state)
                elif heurisitc ==2:
                    f_score[tuple(map(tuple, next_state))] = next_g + euclidean_distance(next_state, goal_state)
                heapq.heappush(heap, (f_score[tuple(map(tuple, next_state))], next_state))
    return None


def DFS(initial_state, goal_state):

    visited = set()
    stack = deque()
    parents = dict()
    stack.append(initial_state)
    nodes_expanded = 0
    max_depth = 0
    start_time = time.time()
    while stack:
        current_state = stack.pop()
        nodes_expanded += 1
        if current_state == goal_state:
            path = []
            while current_state != initial_state:
                path.append(current_state)
                current_state = parents[tuple(map(tuple, current_state))]
            path.append(initial_state)
            path.reverse()
            end_time = time.time()
            return path, nodes_expanded, end_time - start_time, len(path) - 1, max_depth
        visited.add(tuple(map(tuple, current_state)))
        for next_state in get_neighbors(current_state):
            if tuple(map(tuple, next_state)) not in visited:
                parents[tuple(map(tuple, next_state))] = current_state
                stack.append(next_state)
                max_depth = max(max_depth, len(stack))
    return None, nodes_expanded, time.time() - start_time, float('inf'), max_depth


def bfs(start_state, goal_state):
    visited = set()
    queue = Queue()
    queue.put((start_state, []))
    nodes_expanded = 0
    max_depth = 0
    start_time = time.time()
    
    while not queue.empty():
        state, path = queue.get()
        if state == goal_state:
            return path, nodes_expanded, time.time() - start_time, len(path)-1, max_depth
        
        visited.add(tuple(map(tuple, state)))
        for successor in get_neighbors(state):
            if tuple(map(tuple, successor)) not in visited:
                visited.add(tuple(map(tuple, successor)))
                queue.put((successor, path + [successor]))
                max_depth = max(max_depth, len(path) + 1)
                nodes_expanded += 1
                
    return None, nodes_expanded, time.time() - start_time, float('inf'), max_depth




def get_successors(state):
    successors = []
    zero_index = state.index(0)
    if zero_index not in [0, 1, 2]:  # can move up
        new_state = state[:]
        new_state[zero_index], new_state[zero_index - 3] = new_state[zero_index - 3], new_state[zero_index]
        successors.append(new_state)
    if zero_index not in [6, 7, 8]:  # can move down
        new_state = state[:]
        new_state[zero_index], new_state[zero_index + 3] = new_state[zero_index + 3], new_state[zero_index]
        successors.append(new_state)
    if zero_index not in [0, 3, 6]:  # can move left
        new_state = state[:]
        new_state[zero_index], new_state[zero_index - 1] = new_state[zero_index - 1], new_state[zero_index]
        successors.append(new_state)
    if zero_index not in [2, 5, 8]:  # can move right
        new_state = state[:]
        new_state[zero_index], new_state[zero_index + 1] = new_state[zero_index + 1], new_state[zero_index]
        successors.append(new_state)
    return successors




def is_solvable(initial_state, goal_state):
    def count_inversions(lst):
        inversions = 0
        for i in range(len(lst)):
            for j in range(i+1, len(lst)):
                if lst[i] > lst[j] and lst[i] != 0 and lst[j] != 0:
                    inversions += 1
        return inversions

    # flatten the puzzle into a single list
    initial_list = [tile for row in initial_state for tile in row]
    goal_list = [tile for row in goal_state for tile in row]

    # calculate the inversion counts
    initial_inversions = count_inversions(initial_list)
    goal_inversions = count_inversions(goal_list)

    # check if the puzzle is solvable
    if initial_inversions % 2 == goal_inversions % 2:
        return True
    else:
        return False


def is_correct_input(input):
    if len(input) != 9:  # Check if the length of the string is exactly 9
        print("Invalid input: the string must contain exactly 9 digits")
        return False
    elif not all(d.isdigit() for d in input):  # Check if all characters are digits
        print("Invalid input: the string must contain only digits")
        return False
    elif len(set(input)) != 9:  # Check if there are any duplicates
        print("Invalid input: the string must contain only unique digits")
        return False
    else:
        print("Valid input")
        return True

def string_to_state(input):
    lst = [int(i) for i in input]

    # Reshape the list into a 2D list
    initial_state = [lst[i:i + 3] for i in range(0, 9, 3)]
    return initial_state


def display_puzzle(puzzle):
    pygame.init()
    screen = pygame.display.set_mode((300, 300))
    pygame.display.set_caption("Sudoku Puzzle")
    font = pygame.font.Font(None, 36)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        
        screen.fill((255, 255, 255))
        for row in range(9):
            for col in range(9):
                value = puzzle[row][col]
                if value != 0:
                    text = font.render(str(value), True, (0, 0, 0))
                    rect = text.get_rect()
                    rect.center = (col * 33 + 18, row * 33 + 18)
                    screen.blit(text, rect)
        pygame.display.flip()



initial_state1 = [
    [1, 2, 3],
    [0, 4, 6],
    [7, 5, 8]
]

initial_state2 = [
    [0, 8, 7],
    [6, 5, 4],
    [3, 2, 1]
]

initial_state3 = [
    [4, 5, 7],
    [6, 0, 2],
    [1, 3, 8]
]

goal_state = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]

initial_state = []

input_choice = input("Choose input type :\n1.hardcoded input for testing\n2.user input\n")
input_choice = int(input_choice)

if input_choice == 1:
    initial_state = initial_state1
elif  input_choice == 2:

    while True:
        user_input=input("Please enter the 8 puzzle string order\n")
        if is_correct_input(user_input):
            initial_state = string_to_state(user_input)
            break
        else:
            print("please try again")





if is_solvable(initial_state, goal_state):
    print("The puzzle is solvable!")
else:
    print("The puzzle is unsolvable.")
    exit()

algorithm_choice = input("Please enter choose which option you want (1 or 2 or 3) ?\n1.BFS\n2.DFS\n3.A*\n")
algorithm_choice = int(algorithm_choice)

#BFS
if  algorithm_choice == 1:
    path, nodes_expanded, running_time, cost, max_depth = bfs(initial_state, goal_state)
    print("Initial State:")
    for row in initial_state:
        print(row)
    print("\nPath:")
    for i, state in enumerate(path):
        if i <= 4 or i == len(path)-1:
            print("Step", i)
            for row in state:
                print(row)
            print()
        #k=i     
    print("Total steps:", cost)
    print("Nodes expanded:", nodes_expanded)
    print("Running time:", running_time)
    print("Max depth of the tree:", max_depth)


#DFS
elif algorithm_choice == 2:
    path, nodes_expanded, running_time, cost, max_depth = DFS(initial_state, goal_state)
    print("Initial State:")
    for row in initial_state:
        print(row)
    print("\nPath:")
    for i, state in enumerate(path):
        if i <= 4 or i == len(path)-1:
            print("Step", i)
            for row in state:
                print(row)
            print()
    print("Total steps:", cost)
    print("Nodes expanded:", nodes_expanded)
    print("Running time:", running_time)
    print("Max depth of the tree:", max_depth)




#A*
elif algorithm_choice ==3:

    heuristic_choice = input("Please enter choose which option you want (1 or 2) ?\n1.Manhattan Distance\n2.Euclidean Distance\n")
    heuristic_choice = int(heuristic_choice)

    if heuristic_choice == 1:
        start_time = time.time()
        path, cost, nodes_expanded = A_star(initial_state, goal_state,1)
        end_time = time.time()
        time_taken = end_time - start_time

        print("Initial State:")
        for row in initial_state:
            print(row)
        print("\nPath:")
        for i, state in enumerate(path):
            #if i <= 4 or i == len(path) - 1:
                print("Step", i)
                for row in state:
                    print(row)
                print()

        print("Cost of path:", cost)
        print("Number of nodes expanded:", nodes_expanded)
        print("Depth of tree:", len(path) - 1)
        print("Running Time:", time_taken)

    elif heuristic_choice == 2:
        start_time = time.time()
        path, cost, nodes_expanded = A_star(initial_state, goal_state,2)
        end_time = time.time()
        time_taken = end_time - start_time

        print("Initial State:")
        for row in initial_state:
            print(row)
        print("\nPath:")
        for i, state in enumerate(path):
            if i <= 4 or i == len(path) - 1:
                print("Step", i)
                for row in state:
                    print(row)
                print()

        print("Cost of path:", cost)
        print("Number of nodes expanded:", nodes_expanded)
        print("Depth of tree:", len(path) - 1)
        print("Running Time:", time_taken)

else:
    print("invalid choice")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Choose input type :
1.hardcoded input for testing
2.user input
1
The puzzle is solvable!
Please enter choose which option you want (1 or 2 or 3) ?
1.BFS
2.DFS
3.A*
1
Initial State:
[1, 2, 3]
[0, 4, 6]
[7, 5, 8]

Path:
Step 0
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 1
[1, 2, 3]
[4, 6, 0]
[7, 5, 8]

Step 2
[1, 2, 0]
[4, 6, 3]
[7, 5, 8]

Step 3
[1, 0, 2]
[4, 6, 3]
[7, 5, 8]

Step 4
[0, 1, 2]
[4, 6, 3]
[7, 5, 8]

Step 20
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Total steps: 20
Nodes expanded: 81864
Running time: 5.314976930618286
Max depth of the tree: 22
